I'm calling SNPs across all samples, so need to use list of all bamfiles

In order to determine how to filter the sites, need to calculate read depth. 

Therkildsen and Palumbi: excluding sites with a total read depth <300 and >3028 (mean depth +2 standard deviations

So need to calculate the mean total read depth (over all samples). 

Doing some other QC just to see

In [ ]:
~/angsd/angsd -P 15 -b /local/home/sarahgw/WGS/Sequences/filelists/All_Mylu.filelist -ref /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa -out /local/home/sarahgw/WGS/SNPS/All_mylu/All_mylu.qc -trim 0 -C 50 -baq 1 -minMapQ 20 -doQsDist 1 -doDepth 1 -doCounts 1 

In [ ]:
~/angsd/angsd -P 15 -b /local/home/sarahgw/WGS/Sequences/filelists/All_Mylu.filelist -ref /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa -out /local/home/sarahgw/WGS/SNPS/All_mylu/All_mylu_depth.qc -trim 0 -C 50 -baq 1 -minMapQ 20 -doCounts 1 -doDepth 1 

then call SNPs with filters determined above across all samples. only use sites that have information for at least half the individuals

Trying multiple methods for allele frequency estimation just to compare these to the individual population allele freqs. 

**If nina responds that she determined posterior genotype probabilities all together not using the sfs as a prior, you could combine these steps**

In [ ]:
~/angsd/angsd -b /local/home/sarahgw/WGS/Sequences/filelists/All_Mylu.filelist -GL 1 -doMajorMinor 1 -doMaf 11 -minQ 20 -minMapQ 30 -P 10 -SNP_pval 1e-6 -minInd 68 -out /local/home/sarahgw/WGS/SNPS/All_mylu/All_mylu

look for associations for each group.
Paper on method for determining associations: https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-12-231

In [ ]:
sed -e 's/NJ_E_.._pairedplus_marked_dupes.bam/1/g' -e 's/NJ_U_.._pairedplus_marked_dupes.bam/0/g' NJ.filelist > NJ_pheno.ybin

In [ ]:
sed -e 's/NY_E_.._pairedplus_marked_dupes.bam/1/g' -e 's/NY_U_.._pairedplus_marked_dupes.bam/0/g' NY.filelist > NY_pheno.ybin

In [ ]:
sed -e 's/VT_E_.._pairedplus_marked_dupes.bam/1/g' -e 's/VT_U_.._pairedplus_marked_dupes.bam/0/g' VT.filelist > VT_pheno.ybin

In [ ]:
!awk {print} /local/home/sarahgw/WGS/Sequences/filelists/VT_pheno.ybin

In [ ]:
!wc -l /local/home/sarahgw/WGS/Sequences/filelists/NJ_pheno.ybin

In [ ]:
/local/home/sarahgw/angsd/angsd -yBin /local/home/sarahgw/WGS/Sequences/filelists/NJ_pheno.ybin -doAsso 1 -GL 1 -out NJ_assoc -doMajorMinor 1 -doMaf 1 -minQ 20 -minMapQ 30 -SNP_pval 1e-6 -bam /local/home/sarahgw/WGS/Sequences/filelists/NJ.filelist -P 15

In [ ]:
/local/home/sarahgw/angsd/angsd -yBin /local/home/sarahgw/WGS/Sequences/filelists/NY_pheno.ybin -doAsso 1 -GL 1 -out NY_assoc -doMajorMinor 1 -doMaf 1 -minQ 20 -minMapQ 30 -SNP_pval 1e-6 -bam /local/home/sarahgw/WGS/Sequences/filelists/NY.filelist -P 15

In [ ]:
/local/home/sarahgw/angsd/angsd -yBin /local/home/sarahgw/WGS/Sequences/filelists/VT_pheno.ybin -doAsso 1 -GL 1 -out VT_assoc -doMajorMinor 1 -doMaf 1 -minQ 20 -minMapQ 30 -SNP_pval 1e-6 -bam /local/home/sarahgw/WGS/Sequences/filelists/VT.filelist -P 15; echo "VT Assoc done amphiprion" | mailx gw.sarah@gmail.com

ok so if this is true:
The likelihood ratio statistic. This statistic is chi square distributed with one degree of freedom
Then that means p less than 0.05 is greater than 3.84

Right, but you need to do a p adjustment to deal with false positives. Ok, so let's take it into R and calculate p value and correct. 
Wait, maybe this is already accounting for false positives, paper seems to indicate that, no mention of correcting? Oh, so for standard GWAS there is a set significance thresholde of 5 x 10 ^ -8

In [ ]:
!gunzip /local/home/sarahgw/WGS/SNPS/All_mylu/NJ_assoc.lrt0.gz
!gunzip /local/home/sarahgw/WGS/SNPS/All_mylu/NJ_assoc.mafs.gz
!gunzip /local/home/sarahgw/WGS/SNPS/All_mylu/NY_assoc.lrt0.gz
!gunzip /local/home/sarahgw/WGS/SNPS/All_mylu/VT_assoc.lrt0.gz

Going to just try calculating sample allele frequency for snps for two populations, then I can maybe just look for highest changes in allele frequency like they did in tasmanian devil paper: https://www.nature.com/articles/ncomms12684#methods
Fitst, need to make indexes from list of snps so that you can input them as -sites file.

In [ ]:
!gunzip /local/home/sarahgw/WGS/SNPS/All_mylu/VT_assoc.mafs.gz

In [ ]:
!gunzip /local/home/sarahgw/WGS/SNPS/All_mylu/NY_assoc.mafs.gz

In [ ]:
!gunzip /local/home/sarahgw/WGS/SNPS/All_mylu/NJ_assoc.mafs.gz

In [ ]:
!cut -f1-2 /local/home/sarahgw/WGS/SNPS/All_mylu/VT_assoc.mafs > /local/home/sarahgw/WGS/SNPS/All_mylu/VT_SNP_sites.txt

In [ ]:
!cut -f1-2 /local/home/sarahgw/WGS/SNPS/All_mylu/NY_assoc.mafs > /local/home/sarahgw/WGS/SNPS/All_mylu/NY_SNP_sites.txt

In [ ]:
!cut -f1-2 /local/home/sarahgw/WGS/SNPS/All_mylu/NJ_assoc.mafs > /local/home/sarahgw/WGS/SNPS/All_mylu/NJ_SNP_sites.txt

In [ ]:
!head /local/home/sarahgw/WGS/SNPS/All_mylu/NJ_SNP_sites.txt

In [ ]:
!tail -n+2 /local/home/sarahgw/WGS/SNPS/All_mylu/VT_SNP_sites.txt > /local/home/sarahgw/WGS/SNPS/All_mylu/VT_SNP_sites1.txt

In [ ]:
!tail -n+2 /local/home/sarahgw/WGS/SNPS/All_mylu/NY_SNP_sites.txt > /local/home/sarahgw/WGS/SNPS/All_mylu/NY_SNP_sites1.txt

In [ ]:
!tail -n+2 /local/home/sarahgw/WGS/SNPS/All_mylu/NJ_SNP_sites.txt > /local/home/sarahgw/WGS/SNPS/All_mylu/NJ_SNP_sites1.txt

In [ ]:
sort -k1 VT_SNP_sites1.txt >VT_SNP_sites1_sorted.txt

In [ ]:
~/angsd/angsd sites index VT_SNP_sites1.txt

In [ ]:
~/angsd/angsd sites index NY_SNP_sites1.txt

In [ ]:
~/angsd/angsd sites index NJ_SNP_sites1.txt

In [ ]:
~/angsd/angsd sites index NJ_SNP_sites1_sorted.txt

In [ ]:
/local/home/sarahgw/angsd/misc/realSFS print NJ_E.saf.idx NJ_U.saf.idx -r GL429767:6000000-6000100 > NJSaf_test

In [ ]:
!awk '{print NF; exit}' /local/home/sarahgw/WGS/SNPS/All_mylu/NJSaf_test 

Let's try doing doMaf-0 to see if that will give us allele frequencies? No, there is no output

In [ ]:
~/angsd/angsd -b /local/home/sarahgw/WGS/Sequences/filelists/NJ.filelist -GL 1 -doMajorMinor 1 -doMaf 0 -minQ 20 -minMapQ 30 -P 10 -out /local/home/sarahgw/WGS/SNPS/All_mylu/NJ_freq

Okay, now let's calculate the posterior genotype probabilities In order to do that we first need to do sfs with E and U separately. Since I'm not doing the whole thing, need to take first line make new sfs file, remember folded sfs has half as many categories as unfolded sfs

In [ ]:
~/angsd/misc/realSFS -P 20 NJ_E.saf.idx -nsites 1000000000 > NJ_E_sub.ml

In [ ]:
~/angsd/misc/realSFS -P 20 NJ_U.saf.idx -nsites 1000000000 > NJ_U_sub.ml

In [ ]:
./angsd -dopost 3 -b list -gl 1 -domajorminor 1 -domaf 1 -pest angsdput.saf.idx.ml -dogeno 2 -r 1 -out angsdput2

In [ ]:
~/angsd/angsd -dopost 3 -fold 1 -anc /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa -P 10 -b /local/home/sarahgw/WGS/Sequences/filelists/NJ_U.filelist 
-gl 1 -domajorminor 1 -domaf 1 -pest /local/home/sarahgw/WGS/SNPS/All_mylu/NJ_U_sub1.ml -minMapQ 30 -minQ 20 -dogeno 2 -out NJ_U_posterior

In [ ]:
~/angsd/angsd -P 10 -b /local/home/sarahgw/WGS/Sequences/filelists/NJ_E.filelist -fold 1 -anc /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myotis_lucifugus.Myoluc2.0.dna_sm.toplevel.fa -dopost -sfs NJ_U_sub.ml -gl 1 -minMapQ 30 -minQ 20 -doMajorMinor 1 -out NJ_E_posterior